# Sistema RAG con Grafo de Conocimiento

Este notebook implementa un sistema de Generación Aumentada por Recuperación (RAG) usando un grafo de conocimiento pre-construido para responder preguntas sobre lenguajes de programación, frameworks y conceptos tecnológicos.

## Importar Librerías Requeridas



In [ ]:
import json
import pickle
import re
from pathlib import Path
from typing import Dict, List

import networkx as nx
import openai
from dotenv import load_dotenv
import os

## Configurar Variables de Entorno

In [ ]:
load_dotenv(override=True)

# Azure OpenAI API
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key=os.environ["AZURE_OPENAI_API_KEY_US"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT_US"]
)
CHAT_DEPLOYMENT = os.environ["AZURE_OPENAI_DEPLOYMENT_CHAT"]

print(f"Modelo configurado: {CHAT_DEPLOYMENT}")

## Cargar Grafo de Conocimiento Serializado

Cargar el grafo de conocimiento pre-construido desde disco para acceso instantáneo. El grafo contiene entidades y relaciones sobre conceptos tecnológicos.

In [ ]:
# Load graph from pickle
graph_pickle_path = Path("data/knowledge_graph.pkl")

print("Cargando grafo serializado...")
with open(graph_pickle_path, 'rb') as f:
    graph = pickle.load(f)

# Extract entity dictionary
entities = graph.graph['entities']

print(f"Grafo cargado:")
print(f"  - Entidades: {graph.number_of_nodes()}")
print(f"  - Relaciones: {graph.number_of_edges()}")
print(f"  - Tipos de entidad: {set(data['type'] for _, data in graph.nodes(data=True))}")

## Definir Funciones de Búsqueda y Contexto

Implementar búsqueda de entidades con puntuación basada en nombre, tipo y propiedades. También definir funciones para recuperar el contexto de entidades incluyendo relaciones.

In [ ]:
def search_entities(query: str, top_k: int = 5) -> List[Dict]:
    """Search entities by name or properties."""
    query_lower = query.lower()
    query_tokens = [token for token in re.split(r"\W+", query_lower) if token]
    
    if not query_tokens:
        return []
    
    matches = []
    for entity_id, entity in entities.items():
        score = 0
        
        # Name matching
        entity_name = entity['name'].lower()
        name_matches = sum(1 for token in query_tokens if token in entity_name)
        score += 10 * name_matches
        
        # Type matching
        entity_type = entity['type'].lower()
        type_matches = sum(1 for token in query_tokens if token in entity_type)
        score += 5 * type_matches
        
        # Property matching
        for key, value in entity.get('properties', {}).items():
            key_lower = str(key).lower()
            value_lower = str(value).lower()
            prop_matches = sum(
                1 for token in query_tokens 
                if token in key_lower or token in value_lower
            )
            score += 3 * prop_matches
        
        if score > 0:
            matches.append({'entity': entity, 'score': score})
    
    matches.sort(key=lambda x: x['score'], reverse=True)
    return [m['entity'] for m in matches[:top_k]]


def get_entity_context(entity_id: str) -> Dict:
    """Get entity and its immediate relationships."""
    if entity_id not in entities:
        return None
    
    context = {
        'entity': entities[entity_id],
        'relationships': []
    }
    
    # Outgoing relationships
    for target in graph.successors(entity_id):
        edge_data = graph.get_edge_data(entity_id, target)
        context['relationships'].append({
            'type': edge_data['rel_type'],
            'direction': 'outgoing',
            'target': entities[target],
            'properties': edge_data.get('properties', {})
        })
    
    # Incoming relationships
    for source in graph.predecessors(entity_id):
        edge_data = graph.get_edge_data(source, entity_id)
        context['relationships'].append({
            'type': edge_data['rel_type'],
            'direction': 'incoming',
            'source': entities[source],
            'properties': edge_data.get('properties', {})
        })
    
    return context


def format_context_for_llm(contexts: List[Dict]) -> str:
    """Format graph contexts for LLM consumption."""
    if not contexts:
        return "No se encontró información relevante en el grafo de conocimiento."
    
    formatted = "Información del Grafo de Conocimiento:\n\n"
    
    for i, ctx in enumerate(contexts, 1):
        entity = ctx['entity']
        formatted += f"Entidad #{i}: {entity['name']} ({entity['type']})\n"
        
        # Properties
        if entity.get('properties'):
            formatted += "  Propiedades:\n"
            for key, value in entity['properties'].items():
                formatted += f"    - {key}: {value}\n"
        
        # Relationships
        if ctx['relationships']:
            formatted += "  Relaciones:\n"
            for rel in ctx['relationships']:
                if rel['direction'] == 'outgoing':
                    formatted += f"    - {rel['type']} -> {rel['target']['name']} ({rel['target']['type']})\n"
                else:
                    formatted += f"    - {rel['type']} <- {rel['source']['name']} ({rel['source']['type']})\n"
                
                # Relationship properties
                if rel.get('properties'):
                    for key, value in rel['properties'].items():
                        formatted += f"      {key}: {value}\n"
        
        formatted += "\n"
    
    return formatted

print("Funciones de búsqueda definidas")

## Definir Mensajes de Sistema para el LLM

In [ ]:
QUERY_REWRITE_SYSTEM_MESSAGE = """
Eres un asistente útil que reescribe preguntas de usuarios en consultas de palabras clave
para buscar en un grafo de conocimiento sobre tecnología, lenguajes de programación, frameworks y conceptos.

Extrae las entidades clave, conceptos o temas sobre los que pregunta el usuario.
Enfócate en nombres específicos de lenguajes, frameworks, librerías, organizaciones o conceptos técnicos.

Responde SOLO con la consulta de palabras clave (2-6 palabras).
"""

SYSTEM_MESSAGE = """
Eres un asistente útil que responde preguntas usando un grafo de conocimiento sobre
tecnología, lenguajes de programación, frameworks, librerías y conceptos relacionados.

Debes basar tus respuestas en los datos del grafo de conocimiento proporcionados en el contexto.
Si la información no está en el grafo de conocimiento, indícalo claramente.
Usa las relaciones y propiedades para proporcionar respuestas completas y precisas.
"""

## Función de Respuesta a Preguntas

Definir una función para procesar preguntas individuales a través del pipeline RAG: reescritura de consulta, búsqueda de entidades, recuperación de contexto y generación de respuesta.

In [ ]:
def ask_question(question: str) -> str:
    """Ask a single question to the RAG system."""
    messages_local = [{"role": "system", "content": SYSTEM_MESSAGE}]
    
    # Rewrite query
    response = client.chat.completions.create(
        model=CHAT_DEPLOYMENT,
        messages=[
            {"role": "system", "content": QUERY_REWRITE_SYSTEM_MESSAGE},
            {"role": "user", "content": f"Nueva pregunta del usuario: {question}"},
        ],
    )
    search_query = response.choices[0].message.content
    
    # Search and get context
    found_entities = search_entities(search_query, top_k=5)
    
    if found_entities:
        contexts = [get_entity_context(e['id']) for e in found_entities if get_entity_context(e['id'])]
        graph_context = format_context_for_llm(contexts)
    else:
        graph_context = "No se encontró información relevante."
    
    print(f"\n[Consulta de búsqueda]:\n{search_query}")
    print(f"\n[Contexto]:\n{graph_context}")

    # Generate answer
    messages_local.append({
        "role": "user",
        "content": f"{question}\n\nContexto:\n{graph_context}"
    })
    
    response = client.chat.completions.create(
        model=CHAT_DEPLOYMENT,
        messages=messages_local
    )
    
    return response.choices[0].message.content

## Ejemplos de prompts



In [ ]:
# Example usage: Query about Python's use cases
answer = ask_question("¿Para qué se usa Python?")
print(f"\n[Respuesta]:\n{answer}")

In [ ]:
# Example usage: Compare web frameworks
answer = ask_question("Háblame sobre GPT")
print(f"\n[Respuesta]:\n{answer}")

In [ ]:
# Example usage: Query about relationships between technologies
answer = ask_question("¿Qué frameworks están construidos con JavaScript y para qué se usan?")
print(f"\n[Respuesta]:\n{answer}")